In [3]:
import tensorflow as tf
import glob, os, time
import numpy as np
import pandas as pd

In [4]:
!pip install tensorflow

     |███████████████████████████▊    | 442.8 MB 98.0 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 511.7 MB 644 bytes/s 
     |████████████████████████████████| 2.4 MB 51.2 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 4.4 MB 124.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 54.9 MB/s eta 0:00:01
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     |████████████████████████████████| 14.5 MB 57.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 124.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 85.3 MB/s eta 0:00:01
     |████████████████████████████████| 438 kB 144.4 MB/s eta 0:00:01
     |█████████████████████████████

In [4]:
files = glob.glob('./News_text/*')
for i in files:
    text = i.split()[-1].split('.csv')[0]
    df = pd.read_csv(i)
print(i)

./News_text/09211340緊急事態宣言　全面解除視野に検討　28日に決定へ.csv


In [5]:
text = ''
for i in df['コメント']:
    text+=i.replace("\n","")
    text += '\n'
text = text.replace("\u3000","")

In [6]:
print ('Length of text: {} characters'.format(len(text)))

Length of text: 69800 characters


In [7]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

1553 unique characters


In [8]:
# それぞれの文字からインデックスへの対応表を作成
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '%' :   3,
  '&' :   4,
  '(' :   5,
  ')' :   6,
  '+' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '/' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  ...
}


In [10]:
# テキストの最初の 13 文字がどのように整数に変換されるかを見てみる
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'感染者数の視点からはトレン' ---- characters mapped to int ---- > [ 695  859 1183  782  121 1267  978   87  147  122  192  226  229]


In [11]:
# ひとつの入力としたいシーケンスの文字数としての最大の長さ
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# 訓練用サンプルとターゲットを作る
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

感
染
者
数
の


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'感染者数の視点からはトレンドが減少方向ということを含め、十分解除可能だと思われます。病床の方はどうでしょう。入院すべき人が入院できない状況がどうなっているかが気になります。\nもうオミは無視して決めろよ。'
'もう2回打ったし打った方が良いのなら3回目も打つし行動制限なんてもうええ加減にしろよ。\n毎回遅いねん。２日前に決められても困るしアメリカ行く前に決めろよな…完全に解除なら、通常営業に向けて準備とかあるか'
'ら色々困るんですけど一年以上たってまだわからないんですかね？自分らギリギリの生活した事ないからこんな事出来るんでしょうねー。\n５０才以下のワクチン接種率が５割を超えるまで、解除しない方が良いよ。今の流れ'
'は、４波が終わった頃と似ている。その後１か月足らずで、５波が始まったのを、もう忘れてるね。\nまんえん防止を始めちゃうと，分科会がぐずぐずいうのと，それにまして，知事が調子に乗ってちょっと増えてきたときに'
'国費狙いで特に何の思想もなくお金くれ責任は国に預けるよで緊急事態宣言を要請してきます（特に東京大阪はこれやる，これが現状の社会不安の原因です）。離島であることを考慮し沖縄には対策慎重になるとしても他は解'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '感染者数の視点からはトレンドが減少方向ということを含め、十分解除可能だと思われます。病床の方はどうでしょう。入院すべき人が入院できない状況がどうなっているかが気になります。\nもうオミは無視して決めろよ'
Target data: '染者数の視点からはトレンドが減少方向ということを含め、十分解除可能だと思われます。病床の方はどうでしょう。入院すべき人が入院できない状況がどうなっているかが気になります。\nもうオミは無視して決めろよ。'


In [15]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 695 ('感')
  expected output: 859 ('染')
Step    1
  input: 859 ('染')
  expected output: 1183 ('者')
Step    2
  input: 1183 ('者')
  expected output: 782 ('数')
Step    3
  input: 782 ('数')
  expected output: 121 ('の')
Step    4
  input: 121 ('の')
  expected output: 1267 ('視')


In [16]:
# バッチサイズ
BATCH_SIZE = 64

# データセットをシャッフルするためのバッファサイズ
# （TF data は可能性として無限長のシーケンスでも使えるように設計されています。
# このため、シーケンス全体をメモリ内でシャッフルしようとはしません。
# その代わりに、要素をシャッフルするためのバッファを保持しています）
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [17]:
# 文字数で表されるボキャブラリーの長さ
vocab_size = len(vocab)

# 埋め込みベクトルの次元
embedding_dim = 256

# RNN ユニットの数
rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential(
        [tf.keras.layers.Embedding(vocab_size,
                                   embedding_dim,
                                   batch_input_shape=[batch_size, None]),
         tf.keras.layers.GRU(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
         tf.keras.layers.Dense(vocab_size)
        ])
    return model

In [19]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 1553) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           397568    
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 1553)          1591825   
                                                                 
Total params: 5,927,697
Trainable params: 5,927,697
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
sampled_indices

array([ 365,  550, 1236,   97, 1195,  242,  425,  946,  318, 1123,  432,
         44,   60,  413,  164,  941,  533, 1303, 1042, 1245, 1042,  261,
       1101,  560,  363,   83, 1324, 1150,  497, 1200,  816, 1109,  846,
       1033,  525,  899, 1079,  462, 1023,  973,  460,  240, 1495,  347,
       1485,  714, 1431,  242,   17,  140,  587,  926, 1181,  348,  558,
       1364,   85,  872,  232,  965,  808,  739, 1497, 1498,  404,  479,
       1157, 1454,  472,  381, 1138, 1360,   82,  711, 1442,  791,  171,
       1224,  623, 1103,  948, 1368,  617, 1388, 1458, 1205,  975, 1407,
       1043, 1047,  218,  259, 1504,  315,  405, 1134,  514,  132, 1420,
        399])

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '分可能なはずだ。そこまで持っていく前の活動再開は、いい結果に向かわない。\nやっとだな\nやっとだな\n今までと同じ様な繰り返しにならないといいけど。\n全面解除？？段階的じゃなくて？\n地味ん新総裁誕生記念解'

Next Char Predictions: 
 '切完蔑さ肺不去深倣糸取d…卓オ海始請発衆発事立宰出ぇ貰綱堂脅晩等村疲奈殖禍哀町濃和上駄入飛所閥不5も屋況老全室近ぉ森・演映括駆騒包回線雇喫剥終辺う戯陰新ケ花底端添述幹運電腐火酬白盆ャ予高倒化素夕ぺ鎌務'


In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 1553)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       7.347842


In [26]:
model.compile(optimizer='adam', loss=loss)

In [27]:
# チェックポイントが保存されるディレクトリ
checkpoint_dir = './training_checkpoints'
# チェックポイントファイルの名称
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
EPOCHS=30

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
10/10 [==============================] - 7s 563ms/step - loss: 6.4631
Epoch 2/30
10/10 [==============================] - 6s 556ms/step - loss: 5.4794
Epoch 3/30
10/10 [==============================] - 6s 554ms/step - loss: 5.3575
Epoch 4/30
10/10 [==============================] - 6s 554ms/step - loss: 5.2106
Epoch 5/30
10/10 [==============================] - 6s 558ms/step - loss: 4.9864
Epoch 6/30
10/10 [==============================] - 6s 554ms/step - loss: 4.7310
Epoch 7/30
10/10 [==============================] - 6s 553ms/step - loss: 4.4641
Epoch 8/30
10/10 [==============================] - 6s 553ms/step - loss: 4.2875
Epoch 9/30
10/10 [==============================] - 6s 560ms/step - loss: 4.1749
Epoch 10/30
10/10 [==============================] - 6s 559ms/step - loss: 4.0091
Epoch 11/30
10/10 [==============================] - 6s 557ms/step - loss: 3.8592
Epoch 12/30
10/10 [==============================] - 6s 558ms/step - loss: 3.7291
Epoch 13/30
10/10 [======

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            397568    
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 1553)           1591825   
                                                                 
Total params: 5,927,697
Trainable params: 5,927,697
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, start_string):
  # 評価ステップ（学習済みモデルを使ったテキスト生成）

  # 生成する文字数
    num_generate = 100

  # 開始文字列を数値に変換（ベクトル化）
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
  # 結果を保存する空文字列
    text_generated = []

  # 低い temperature　は、より予測しやすいテキストをもたらし
  # 高い temperature は、より意外なテキストをもたらす
  # 実験により最適な設定を見つけること
    temperature = 1.0

  # ここではバッチサイズ　== 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # バッチの次元を削除
        predictions = tf.squeeze(predictions, 0)

      # カテゴリー分布をつかってモデルから返された文字を予測 
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 過去の隠れ状態とともに予測された文字をモデルへのつぎの入力として渡す
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model,
                    start_string=u"感染者"))

感染者が減少してから想定すれば良いでしょう
解除したら、急・・宣言」できついたらどうコロナ禍にそんじゃないとわるのがいいんじゃな。
治っ的に全て良いと思ってるだきたないから思い
仕込み直状態陽性者数の確認に


In [34]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [35]:
optimizer = tf.keras.optimizers.Adam()
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [36]:
# 訓練ステップ
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

  # 各エポックの最初に、隠れ状態を初期化する
  # 最初は隠れ状態は None
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

    if batch_n % 100 == 0:
        template = 'Epoch {} Batch {} Loss {}'
        print(template.format(epoch+1, batch_n, loss))

  # 5エポックごとにモデル（のチェックポイント）を保存する
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))



Epoch 1 Loss 5.6404
Time taken for 1 epoch 6.405139207839966 sec

Epoch 2 Loss 5.3975
Time taken for 1 epoch 5.497836112976074 sec

Epoch 3 Loss 5.3376
Time taken for 1 epoch 5.4719154834747314 sec

Epoch 4 Loss 5.1549
Time taken for 1 epoch 5.480853080749512 sec

Epoch 5 Loss 4.9070
Time taken for 1 epoch 5.504909992218018 sec

Epoch 6 Loss 4.6821
Time taken for 1 epoch 5.474076986312866 sec

Epoch 7 Loss 4.4087
Time taken for 1 epoch 5.490312814712524 sec

Epoch 8 Loss 4.1618
Time taken for 1 epoch 5.454525947570801 sec

Epoch 9 Loss 4.0911
Time taken for 1 epoch 5.534043788909912 sec

Epoch 10 Loss 3.8981
Time taken for 1 epoch 5.569545745849609 sec

